In [ ]:
from __future__ import (absolute_import, division, print_function)
from functools import reduce
from operator import mul
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from pyneqsys.symbolic import TransformedSys, linear_exprs
sp.init_printing()
prod = lambda x: reduce(mul, x)
names = 'H+ OH- NH4+ NH3 H2O'.split()

In [ ]:
iHp, iOHm, iNH4p, iNH3, iH2O = init_concs = [sp.Symbol('i_'+str(i), real=True, negative=False) for i in range(5)]
c = Hp, OHm, NH4p, NH3, H2O = [sp.Symbol('c_'+str(i), real=True, negative=False) for i in range(5)]
stoichs = [[1, 1, 0, 0, -1], [1, 0, -1, 1, 0]]
H = [1, 1, 4, 3, 2]
N = [0, 0, 1, 1, 0]
O = [0, 1, 0, 0, 1]
e = [1, -1, 1, 0, 0]
preserv = [H, N, O, e]
eq_constants = Kw, Ka = [sp.Symbol(K, real=True, positive=True) for K in 'K_w K_a'.split()] 
def get_f(x, params):
    init_concs = params[:5]
    eq_constants = params[5:]
    le = linear_exprs(preserv, x, linear_exprs(preserv, init_concs), rref=True)
    return le + [
        sp.Eq(prod(xi**p for xi, p in zip(x, coeffs)), K) for coeffs, K in zip(stoichs, eq_constants)
    ]
get_f(c, init_concs + eq_constants)

In [ ]:
def my_log_transform(expr):
    if isinstance(expr, sp.Eq):
        return sp.expand_log(sp.log(expr.lhs), force=True) - sp.expand_log(sp.log(expr.rhs), force=True)
    else:
        return expr

In [ ]:
tf = [my_log_transform(expr) for expr in get_f(c, init_concs + eq_constants)]
tf

In [ ]:
subs_fw = [(ci, sp.exp(ci)) for ci in c]
subs_bw = [(ci, sp.log(ci)) for ci in c]
ttf = [expr.subs(subs_fw) for expr in tf]
ttf

Now let's see how symneqsys can do this for us:

In [ ]:
neqsys = TransformedSys.from_callback(get_f, 5, 7, my_log_transform, (sp.exp, sp.log))
neqsys.exprs

In [ ]:
#  Hp, OHm, NH4p, NH3, H2O
c0 = 1e-7, 1e-7, 1e-7, 1, 55
K = Kw, Ka = 10**-14/55, 10**-9.24
c_res, sol = neqsys.solve_scipy(c0, np.array(c0+K))
c_res, sol.success

In [ ]:
NH3_varied = np.logspace(-7, 0)
NH3_idx = 3
xres, sols = neqsys.solve_series('scipy', c0, c0+K, NH3_varied, NH3_idx)

In [ ]:
%matplotlib inline
neqsys.plot_series(NH3_idx, NH3_varied, xres)
plt.gca().set_xscale('log'); plt.gca().set_yscale('log');
_ = plt.legend(names, loc='best')